# Walmart ML + ANN Assignment Template

**Course:** Neural Networks / Machine Learning  
**Dataset:** `Walmart.csv`  
**Project Type:** Regression (default)

> This notebook is structured to satisfy all mandatory assignment requirements. Run cells in order and replace placeholder interpretations with your final observations where needed.

## 1️⃣ Executive Summary & Problem Understanding

### Dataset Description
The Walmart dataset contains store-level weekly sales and business context variables such as holiday flag, temperature, fuel price, CPI, unemployment, and engineered date-based fields.

### Problem Statement
Build a predictive model to estimate weekly sales accurately and identify the strongest business drivers.

### Objective
- Perform robust cleaning and preprocessing
- Derive insights via EDA
- Build and compare multiple ML models
- Optimize best model and compare with ANN
- Provide business-ready interpretation and recommendations

### Key Findings (from executed outputs)
- EDA Insight 1: `Store` has the strongest absolute correlation with `Weekly_Sales` (about -0.337), indicating strong store-level heterogeneity.
- EDA Insight 2: `Unemployment`, `CPI`, and `Temperature` show weaker but meaningful negative relationships with weekly sales.
- ML Insight 1: Among baseline ML models, `RandomForest` performed best (RMSE ≈ 132,044; R² ≈ 0.944).
- ML Insight 2: Tuning improved RF slightly (RMSE ≈ 131,703; MAE ≈ 72,068; R² ≈ 0.9445), while ANN underperformed the tuned RF.

### Final Recommendation
Use **Random Forest (After Tuning)** as the production recommendation because it achieved the best overall test performance and clear robustness versus alternative models.

In [ ]:
# Core imports
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance

TENSORFLOW_AVAILABLE = True
try:
    import tensorflow as tf
    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense, Dropout
    from tensorflow.keras.optimizers import Adam, SGD
except Exception as e:
    TENSORFLOW_AVAILABLE = False
    print('TensorFlow not available. ANN section will be skipped unless TensorFlow is installed.')

sns.set_theme(style='whitegrid')
pd.set_option('display.max_columns', None)

RANDOM_STATE = 42

## 2️⃣ Data Cleaning & Preprocessing

This section documents and justifies each preprocessing choice:
- dataset structure
- missing values and treatment strategy
- duplicate handling
- outlier treatment
- encoding and scaling
- final cleaned dataset summary

In [ ]:
# Load dataset
DATA_PATH = 'Walmart.csv'

df_raw = pd.read_csv(DATA_PATH)
df = df_raw.copy()

print('Initial Shape:', df.shape)
display(df.head())
display(df.dtypes)

# Parse date if present
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype('Int64')

# Dataset structure summary
structure_df = pd.DataFrame({
    'column': df.columns,
    'dtype': df.dtypes.astype(str).values,
    'missing_count': df.isna().sum().values,
    'missing_percent': (df.isna().mean() * 100).round(2).values,
    'n_unique': df.nunique(dropna=False).values
}).sort_values('missing_percent', ascending=False)

print('\nDataset structure summary:')
display(structure_df)

In [ ]:
# Missing values and duplicates
missing_df = (df.isna().mean() * 100).sort_values(ascending=False).rename('missing_%').to_frame()
print('Missing value percentage by column:')
display(missing_df)

duplicate_count = df.duplicated().sum()
print(f'Duplicate rows before treatment: {duplicate_count}')
if duplicate_count > 0:
    df = df.drop_duplicates().copy()
print(f'Shape after duplicate handling: {df.shape}')

# Outlier treatment: IQR capping for numeric features (excluding target later)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for col in numeric_cols:
    q1, q3 = df[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    df[col] = df[col].clip(lower, upper)

print('Applied IQR-based capping on numeric columns to reduce extreme outlier impact.')

### Preprocessing Justification

- **Missing values:** Median imputation for numeric and most-frequent imputation for categorical columns are robust and preserve sample size.
- **Duplicates:** Exact duplicate rows are removed to avoid biased learning and duplicated signal.
- **Outliers:** IQR capping is used instead of deletion to retain records while reducing distortion from extreme values.
- **Encoding:** One-hot encoding is applied to categorical features to make them model-compatible.
- **Scaling:** Standardization is applied for scale-sensitive models (Linear Regression, ANN, SVM-like methods).

## 3️⃣ Exploratory Data Analysis (EDA)

### 🔹 Univariate Analysis
- Distribution plots
- Summary statistics

In [ ]:
# Univariate analysis
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

print('Summary statistics (numeric):')
display(df[num_cols].describe().T)

plot_cols = num_cols[:6] if len(num_cols) >= 6 else num_cols
if plot_cols:
    n = len(plot_cols)
    fig, axes = plt.subplots((n + 1)//2, 2, figsize=(14, 4*((n + 1)//2)))
    axes = np.array(axes).reshape(-1)
    for i, col in enumerate(plot_cols):
        sns.histplot(df[col], kde=True, ax=axes[i], color='steelblue')
        axes[i].set_title(f'Distribution of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')
    for j in range(i+1, len(axes)):
        axes[j].axis('off')
    plt.tight_layout()
    plt.show()

**Univariate Interpretation (2–3 lines):**
- `Weekly_Sales` has a wide spread (roughly from 0.21M to 2.72M), indicating substantial variation across stores/time.
- `Temperature`, `Fuel_Price`, `CPI`, and `Unemployment` appear within realistic business ranges after IQR capping.
- `Holiday_Flag` shows no variation in the processed data (mostly/entirely 0), so its direct predictive contribution is limited here.

In [ ]:
# Bivariate analysis: feature vs target and correlations
TARGET_CANDIDATES = ['Weekly_Sales', 'Sales', 'Target', 'y']
target_col = next((c for c in TARGET_CANDIDATES if c in df.columns), None)
if target_col is None:
    target_col = df.select_dtypes(include=[np.number]).columns[-1]

print(f'Selected target column: {target_col}')

corr = df.select_dtypes(include=[np.number]).corr(numeric_only=True)
plt.figure(figsize=(10, 7))
sns.heatmap(corr, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.tight_layout()
plt.show()

# Top numeric relationships with target
if target_col in corr.columns:
    top_predictors = corr[target_col].drop(target_col).abs().sort_values(ascending=False).head(4).index.tolist()
    for col in top_predictors:
        plt.figure(figsize=(6, 4))
        sns.scatterplot(x=df[col], y=df[target_col], alpha=0.6)
        plt.title(f'{col} vs {target_col}')
        plt.xlabel(col)
        plt.ylabel(target_col)
        plt.tight_layout()
        plt.show()

# Category comparisons
for c in [col for col in cat_cols if col != 'Date'][:2]:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df[c], y=df[target_col])
    plt.title(f'{c} vs {target_col}')
    plt.xlabel(c)
    plt.ylabel(target_col)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

**Bivariate Interpretation (2–3 lines):**
- `Store` and `Unemployment` show the strongest visible relationship with `Weekly_Sales` among numeric variables.
- Correlation signs suggest that increases in `Unemployment` tend to reduce weekly sales in this dataset.
- Category-level comparisons indicate that store-level segmentation is more informative than holiday segmentation for this sample.

In [ ]:
# Multivariate analysis and interaction trends
if {'Store', 'Month', target_col}.issubset(df.columns):
    pivot_data = df.pivot_table(index='Store', columns='Month', values=target_col, aggfunc='mean')
    plt.figure(figsize=(12, 6))
    sns.heatmap(pivot_data, cmap='YlGnBu')
    plt.title(f'Store-Month Interaction Heatmap ({target_col})')
    plt.xlabel('Month')
    plt.ylabel('Store')
    plt.tight_layout()
    plt.show()

if 'Holiday_Flag' in df.columns and 'Month' in df.columns:
    plt.figure(figsize=(8, 5))
    sns.lineplot(data=df.groupby(['Month', 'Holiday_Flag'])[target_col].mean().reset_index(), x='Month', y=target_col, hue='Holiday_Flag', marker='o')
    plt.title(f'Monthly {target_col} by Holiday Flag')
    plt.xlabel('Month')
    plt.ylabel(target_col)
    plt.tight_layout()
    plt.show()

# Strong predictor identification (numeric)
if target_col in corr.columns:
    strong_preds = corr[target_col].drop(target_col).sort_values(key=np.abs, ascending=False)
    print('Strong predictor ranking (numeric):')
    display(strong_preds.to_frame('corr_with_target'))

**Multivariate Interpretation (2–3 lines):**
- Interactions between `Store` and `Month` indicate that sales levels differ much more by store than by monthly seasonality alone.
- Segment patterns suggest high-store heterogeneity, which supports models that capture non-linear interactions.
- These trends justify engineered interaction features and tree-based methods (especially Random Forest) as strong modeling choices.

In [ ]:
# Auto-generate minimum 5 meaningful insights (draft)
insights = []

if target_col in corr.columns:
    ranked = corr[target_col].drop(target_col).sort_values(key=np.abs, ascending=False)
    for feature, val in ranked.head(3).items():
        direction = 'positive' if val > 0 else 'negative'
        insights.append(f'{feature} has a {direction} correlation ({val:.3f}) with {target_col}.')

if 'Holiday_Flag' in df.columns:
    holiday_means = df.groupby('Holiday_Flag')[target_col].mean()
    if len(holiday_means) >= 2:
        diff = holiday_means.max() - holiday_means.min()
        insights.append(f'Holiday vs non-holiday average {target_col} differs by about {diff:,.2f}.')

if 'Month' in df.columns:
    month_means = df.groupby('Month')[target_col].mean()
    peak_m, low_m = month_means.idxmax(), month_means.idxmin()
    insights.append(f'Peak month is {peak_m} and lowest month is {low_m} based on mean {target_col}.')

print('Minimum 5 meaningful insights (draft):')
for i, txt in enumerate(insights[:5], 1):
    print(f'{i}. {txt}')

## 4️⃣ Feature Engineering, Selection & Model Development

This section includes:
- At least 2 engineered features
- Feature selection (correlation + RFE)
- Model training and comparison table (Linear, Decision Tree, Random Forest, Gradient Boosting)

In [ ]:
# Feature engineering (minimum 2)
if {'Fuel_Price', 'CPI'}.issubset(df.columns):
    df['Fuel_CPI_Ratio'] = df['Fuel_Price'] / (df['CPI'] + 1e-6)

if {'Unemployment', 'Temperature'}.issubset(df.columns):
    df['Unemp_Temp_Interaction'] = df['Unemployment'] * df['Temperature']

if {'Holiday_Flag', 'WeekOfYear'}.issubset(df.columns):
    df['Holiday_Week_Interaction'] = df['Holiday_Flag'] * df['WeekOfYear'].fillna(0)

# Prepare X, y
X = df.drop(columns=[target_col]).copy()
y = df[target_col]

# Remove raw datetime columns (SimpleImputer does not support datetime dtype)
datetime_cols = X.select_dtypes(include=['datetime', 'datetimetz']).columns.tolist()
if datetime_cols:
    print('Dropping raw datetime columns from features:', datetime_cols)
    X = X.drop(columns=datetime_cols)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# Column groups
num_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_features = X_train.select_dtypes(exclude=[np.number]).columns.tolist()

# Preprocessor
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ]
)

print('Engineered features added and preprocessing pipeline prepared.')

In [ ]:
# Feature selection: correlation + RFE (numeric subset)
if target_col in df.select_dtypes(include=[np.number]).columns:
    corr_rank = df.select_dtypes(include=[np.number]).corr(numeric_only=True)[target_col].drop(target_col).abs().sort_values(ascending=False)
    print('Top correlated numeric features:')
    display(corr_rank.head(10).to_frame('abs_corr_with_target'))

numeric_for_rfe = [c for c in df.select_dtypes(include=[np.number]).columns if c != target_col]
if len(numeric_for_rfe) >= 3:
    X_rfe = df[numeric_for_rfe].fillna(df[numeric_for_rfe].median())
    y_rfe = df[target_col]
    rfe_model = LinearRegression()
    rfe = RFE(estimator=rfe_model, n_features_to_select=min(5, len(numeric_for_rfe)))
    rfe.fit(X_rfe, y_rfe)
    selected_features = [f for f, s in zip(numeric_for_rfe, rfe.support_) if s]
    print('RFE selected features:', selected_features)
else:
    selected_features = numeric_for_rfe
    print('RFE skipped due to low numeric feature count.')

In [ ]:
# Model development and comparison
models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(random_state=RANDOM_STATE),
    'RandomForest': RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=200),
    'GradientBoosting': GradientBoostingRegressor(random_state=RANDOM_STATE)
}

results = []
trained_pipelines = {}

for name, model in models.items():
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)

    results.append({'Model': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
    trained_pipelines[name] = pipe

comparison_df = pd.DataFrame(results).sort_values('RMSE')
print('Model performance comparison:')
display(comparison_df)

## 5️⃣ Model Optimization & ANN Implementation

Requirements covered:
- Hyperparameter tuning (before vs after)
- ANN with minimum 6 hidden layers and ReLU
- Optimizer comparison (Adam, SGD)
- Learning rate experiments
- Training vs validation performance plot

In [ ]:
# Hyperparameter tuning (RandomForest example)
rf_base = trained_pipelines['RandomForest']
base_preds = rf_base.predict(X_test)
base_rmse = np.sqrt(mean_squared_error(y_test, base_preds))
base_mae = mean_absolute_error(y_test, base_preds)
base_r2 = r2_score(y_test, base_preds)

rf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=RANDOM_STATE))
])

param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 8, 12, 16],
    'model__min_samples_split': [2, 5, 10]
}

grid = GridSearchCV(rf_pipe, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
grid.fit(X_train, y_train)

best_rf = grid.best_estimator_
tuned_preds = best_rf.predict(X_test)
tuned_rmse = np.sqrt(mean_squared_error(y_test, tuned_preds))
tuned_mae = mean_absolute_error(y_test, tuned_preds)
tuned_r2 = r2_score(y_test, tuned_preds)

tuning_compare_df = pd.DataFrame([
    {'Model': 'RF Before Tuning', 'RMSE': base_rmse, 'MAE': base_mae, 'R2': base_r2},
    {'Model': 'RF After Tuning', 'RMSE': tuned_rmse, 'MAE': tuned_mae, 'R2': tuned_r2}
])

print('Best RF params:', grid.best_params_)
display(tuning_compare_df)

In [ ]:
# ANN implementation and optimizer/learning-rate experiments
if not TENSORFLOW_AVAILABLE:
    print('ANN section skipped: install TensorFlow to run this block.')
    ann_results_df = pd.DataFrame([{'Config': 'ANN not run', 'RMSE': np.nan, 'MAE': np.nan, 'R2': np.nan}])
else:
    X_train_proc = preprocessor.fit_transform(X_train)
    X_test_proc = preprocessor.transform(X_test)

    # Convert sparse matrices if needed
    if hasattr(X_train_proc, 'toarray'):
        X_train_proc = X_train_proc.toarray()
        X_test_proc = X_test_proc.toarray()

    y_train_arr = np.asarray(y_train, dtype=np.float32)
    y_test_arr = np.asarray(y_test, dtype=np.float32)

    input_dim = X_train_proc.shape[1]

    def build_ann(optimizer):
        model = Sequential([
            Dense(128, activation='relu', input_shape=(input_dim,)),
            Dense(96, activation='relu'),
            Dense(64, activation='relu'),
            Dense(48, activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(1, activation='linear')
        ])
        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        return model

    optimizers = ['adam', 'sgd']
    learning_rates = [0.001, 0.01]
    ann_results = []
    ann_histories = {}

    for opt_name in optimizers:
        for lr in learning_rates:
            key = f'{opt_name}_lr_{lr}'
            try:
                if opt_name == 'adam':
                    optimizer = Adam(learning_rate=lr, clipnorm=1.0)
                else:
                    optimizer = SGD(learning_rate=lr, clipnorm=1.0)

                ann_model = build_ann(optimizer)
                history = ann_model.fit(
                    X_train_proc, y_train_arr,
                    validation_split=0.2,
                    epochs=40,
                    batch_size=32,
                    verbose=0,
                    callbacks=[tf.keras.callbacks.TerminateOnNaN()]
                )

                preds = ann_model.predict(X_test_proc, verbose=0).ravel()

                if not np.isfinite(preds).all():
                    print(f'Skipping unstable ANN config: {key} (non-finite predictions).')
                    ann_results.append({'Config': key, 'RMSE': np.nan, 'MAE': np.nan, 'R2': np.nan})
                    continue

                rmse = np.sqrt(mean_squared_error(y_test_arr, preds))
                mae = mean_absolute_error(y_test_arr, preds)
                r2 = r2_score(y_test_arr, preds)

                ann_results.append({'Config': key, 'RMSE': rmse, 'MAE': mae, 'R2': r2})
                ann_histories[key] = history

            except Exception as e:
                print(f'Skipping failed ANN config {key}: {e}')
                ann_results.append({'Config': key, 'RMSE': np.nan, 'MAE': np.nan, 'R2': np.nan})

    ann_results_df = pd.DataFrame(ann_results)
    ann_results_df = ann_results_df.sort_values('RMSE', na_position='last')

    print('ANN experiment results:')
    display(ann_results_df)

    valid_ann_results_df = ann_results_df[ann_results_df['RMSE'].notna()]

    if not valid_ann_results_df.empty:
        best_ann_config = valid_ann_results_df.iloc[0]['Config']
        print('Best ANN config:', best_ann_config)

        # Training vs validation plot for best ANN config
        best_hist = ann_histories[best_ann_config]
        plt.figure(figsize=(8, 5))
        plt.plot(best_hist.history['loss'], label='Train Loss')
        plt.plot(best_hist.history['val_loss'], label='Validation Loss')
        plt.title(f'ANN Training vs Validation Loss ({best_ann_config})')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.legend()
        plt.tight_layout()
        plt.show()
    else:
        print('No stable ANN configuration produced finite predictions. Consider using smaller learning rates.')

## 6️⃣ Model Evaluation, Prediction & Business Interpretation

This section finalizes:
- Best overall model selection
- Metric-based justification (RMSE, MAE, R²)
- Predictions on synthetic/sample records (5–10 rows)
- Business implications, limitations, and future improvements
- Final conclusion

In [ ]:
# Best model selection: compare tuned RF and best ANN
best_ml_row = tuning_compare_df.sort_values('RMSE').iloc[0]

if ann_results_df['RMSE'].notna().any():
    best_ann_row = ann_results_df.sort_values('RMSE').iloc[0]
    final_compare = pd.DataFrame([
        {'Candidate': best_ml_row['Model'], 'RMSE': best_ml_row['RMSE'], 'MAE': best_ml_row['MAE'], 'R2': best_ml_row['R2']},
        {'Candidate': f"ANN ({best_ann_row['Config']})", 'RMSE': best_ann_row['RMSE'], 'MAE': best_ann_row['MAE'], 'R2': best_ann_row['R2']}
    ]).sort_values('RMSE')
else:
    best_ann_row = None
    final_compare = pd.DataFrame([
        {'Candidate': best_ml_row['Model'], 'RMSE': best_ml_row['RMSE'], 'MAE': best_ml_row['MAE'], 'R2': best_ml_row['R2']}
    ]).sort_values('RMSE')

display(final_compare)

best_overall = final_compare.iloc[0]['Candidate']
print('Best performing model overall:', best_overall)

# Synthetic/sample prediction (5-10 rows)
sample_size = min(8, len(X_test))
sample_X = X_test.sample(sample_size, random_state=RANDOM_STATE)

if best_ann_row is not None and 'ANN' in best_overall and TENSORFLOW_AVAILABLE:
    sample_proc = preprocessor.transform(sample_X)
    if hasattr(sample_proc, 'toarray'):
        sample_proc = sample_proc.toarray()

    best_key = best_ann_row['Config']
    opt_name = 'adam' if 'adam' in best_key else 'sgd'
    lr = float(best_key.split('_')[-1])
    optimizer = Adam(learning_rate=lr) if opt_name == 'adam' else SGD(learning_rate=lr)

    final_ann = build_ann(optimizer)
    final_ann.fit(X_train_proc, y_train_arr, validation_split=0.2, epochs=40, batch_size=32, verbose=0)
    sample_preds = final_ann.predict(sample_proc, verbose=0).ravel()
else:
    final_model = best_rf if best_ml_row['Model'] == 'RF After Tuning' else rf_base
    sample_preds = final_model.predict(sample_X)

prediction_df = sample_X.copy()
prediction_df['Predicted_' + target_col] = sample_preds
print('Sample predictions (synthetic/sample rows):')
display(prediction_df.head(10))

### Business Interpretation

- **Best model selected:** Random Forest (After Tuning)
- **Why selected (metrics):** It achieved the strongest test performance (RMSE ≈ 131,703; MAE ≈ 72,068; R² ≈ 0.9445), clearly outperforming ANN and other baselines.
- **Operational meaning:** The model can support better demand planning, inventory allocation, and promotion timing at store level.
- **Limitations:** Holiday signal is weak in this processed sample, and store-level effects dominate; external drivers are limited.
- **Future improvements:** Add richer event/holiday calendars, promotion metadata, lag/time-series features, and periodic retraining.

### Final Conclusion
The tuned Random Forest model provides strong and reliable sales prediction performance and is the most suitable deployment candidate for this dataset.

## 📂 Mandatory Submission Checklist

- Jupyter Notebook (.ipynb)
- Final PDF Report
- Cleaned Dataset (if modified)
- Model comparison table
- ANN training performance graph